In [9]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log

#‘eps’ here is the smallest representable number. 
#At times we get log(0) or 0 in the denominator, 
#to avoid that we are going to use this.

dataset = pd.read_csv('glass1.csv')

#dataset = {'Age':['36-55','18-35','36-55','18-35','<18','18-35','36-55','36-55','36-55','>55','36-55','>55','<18','36-55','36-55','<18','18-35','>55','>55','36-55',],
 #          'Education':['Masters','High School','Masters','Bachelors','High School','Bachelors','Bachelors','Bachelors','Masters','Masters','Masters','Masters','High School','Masters','High School','High School','Bachelors','High School','Bachelors','High School',],
  #         'Income':['High','Low','Low','High','Low','High','Low','High','Low','Low','High','High','High','Low','Low','Low','High','High','Low','High',],
   #        'Marrital_Status':['Single','Single','Single','Single','Single','Married','Married','Single','Married','Married','Single','Single','Single','Single','Single','Married','Married','Married','Single','Married',],
    #       'Buy_Computer': ['Yes','No','Yes','No','Yes','No','No','Yes','No','Yes','Yes','Yes','No','Yes','Yes','Yes','No','Yes','Yes','No',]
     #   }

#df = pd.DataFrame(dataset,columns=['Age','Education','Income','Marrital_Status','Buy_Computer'])

#decision tree
def find_entropy(dataset):
    class_name = dataset.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = dataset[class_name].unique()
    for value in values:
        fraction = dataset[class_name].value_counts()[value]/len(dataset[class_name])
        entropy += -fraction*np.log2(fraction)
    return entropy

def find_entropy_attribute(dataset,attribute):
    class_name = dataset.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = dataset[class_name].unique()  
    variables = dataset[attribute].unique()    #This gives different features in attribute
    entropy2 = 0
    for variable in variables:
        entropy = 0
        for target_variable in target_variables:
            num = len(dataset[attribute][dataset[attribute]==variable][dataset[class_name] ==target_variable])
            den = len(dataset[attribute][dataset[attribute]==variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
        fraction2 = den/len(dataset)
        entropy2 += -fraction2*entropy
    return abs(entropy2)

def find_root(dataset):

    IG = []
    for key in dataset.keys()[:-1]:
        IG.append(find_entropy(dataset)-find_entropy_attribute(dataset,key))
        
    return dataset.keys()[:-1][np.argmax(IG)]
  



In [10]:


def get_subtable(dataset, node,value):
    return dataset[dataset[node] == value].reset_index(drop=True)



def build_tree(dataset,tree=None): 
    class_name = dataset.keys()[-1]   #To make the code generic, changing target variable class name
    
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_root(dataset)
    
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(dataset[node])
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(dataset,node,value)
        clValue,counts = np.unique(subtable[class_name],return_counts=True)                        
        
        if len(counts)==1:                              #Checking purity of subset
            tree[node][value] = clValue[0]                                                    
        else:        
            tree[node][value] = build_tree(subtable)    #Calling the function recursively 
                   
    return tree
tree = build_tree(dataset)
import pprint
pprint.pprint(tree)

{'1.51588824': {1.51115: ' negative',
                1.51130946: ' negative',
                1.51215232: ' negative',
                1.51299518: ' negative',
                1.51315464: ' negative',
                1.5132002: ' negative',
                1.51408862: ' positive',
                1.51509094: ' negative',
                1.5151364999999999: ' negative',
                1.51531874: ' negative',
                1.51545542: ' negative',
                1.51556932: ' negative',
                1.51566044: ' negative',
                1.51568322: ' positive',
                1.515706: ' negative',
                1.51572878: ' positive',
                1.51588824: ' positive',
                1.51591102: ' positive',
                1.5159338: ' positive',
                1.51595658: {' 12.87795': {12.7915: ' negative',
                                           13.0176: ' positive'}},
                1.51602492: ' negative',
                1.5160476999999999: ' positive'